In [1]:
import pickle
import baseline_nel
import knowledge_graph_extraction
import pandas as pd
from tqdm import tqdm

In [2]:
with open('../data/sample_data_candidates.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
len(data)

4988

In [3]:
# db = "../data/kensho.db"

# query = """
# SELECT 
#     source_item_id, edge_property_id, target_item_id
# FROM 
#     wikipage_triplets
# """

# triplets = knowledge_graph_extraction.conduct_sql_query(db, query)

In [4]:
# baseline_nel.predict_local(triplets, data[0][0], data[0][1], top_k)

In [ ]:
predictions = []
triplets_dicts = []
top_k = 10
try:
    for i in tqdm(range(len(data))):
        if i % 500 == 0:
            with open('../data/sample_baseline_predictions_' + str(i) + '.pkl', 'wb') as f:
                pickle.dump(predictions, f)
            with open('../data/sample_baseline_triplets_' + str(i) + '.pkl', 'wb') as f:
                pickle.dump(triplets_dicts, f)
        row = data[i]
        prediction, triplets = baseline_nel.predict("../data/kensho.db", "wikipage_triplets", row[0], row[1], top_k)
        predictions.append(prediction)
        triplets_dicts.append(triplets)
except:
    with open('../data/error_' + str(i), 'wb') as f:
        pickle.dump(['error at '+ str(i)], f)

 88%|████████▊ | 4396/5000 [3:07:33<27:17,  2.71s/it]  

In [ ]:
with open('../data/sample_baseline_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)
with open('../data/sample_baseline_triplets.pkl', 'wb') as f:
    pickle.dump(triplets_dicts, f)

In [4]:
with open('../data/sample_baseline_predictions.pkl', 'rb') as f:
    predictions = pickle.load(f)

In [11]:
predictions_dict = {prediction[0] : prediction[1] for prediction in predictions}
candidates_dict = {candidate[0]: candidate[1] for candidate in data}

In [12]:
def compute_accuracy(ground_truth_dict, prediction_dict, top_k):
    total = 0
    correct = 0
    for k, v in ground_truth_dict.items():
        if top_k is None:
            if k in prediction_dict and v in prediction_dict[k]:
                correct += 1
        else:
            if k in prediction_dict and v in prediction_dict[k][:top_k]:
                correct += 1
        total += 1
    if total == 0:
        return 0, 0
    return correct, total

In [7]:
with open('../data/sample_labels.pkl', 'rb') as f:
    labels = pickle.load(f)

In [8]:
len(labels)

4988

In [13]:
for top_k in [1, 5, 10]:
    total_correct, total_num_labels = 0, 0
    for i, row in enumerate(labels):
        text = row[0]
        prediction= predictions_dict[text]
        prediction = {key[:3]: value for key, value in prediction.items()}
        label = {key[:3]: key[3] for key in row[1].keys()}
        correct, num_labels = compute_accuracy(label, prediction, top_k)
        total_correct += correct
        total_num_labels += num_labels 
    print("Accuracy for Top {}: {}".format(top_k, total_correct / total_num_labels))
    

Accuracy for Top 1: 0.2599408825392357
Accuracy for Top 5: 0.39615736505032023
Accuracy for Top 10: 0.4651981138714899


In [18]:
total_correct, total_num_labels = 0, 0
for i, row in enumerate(labels):
    text = row[0]
    candidate= candidates_dict[text]
    candidate = {key[:3]: value for key, value in candidate.items()}
    label = {key[:3]: key[3] for key in row[1].keys()}
    correct, num_labels = compute_accuracy(label, candidate, None)
    total_correct += correct
    total_num_labels += num_labels 
print("Percent of Correct Wikidata item in Candidates list: {}".format(total_correct / total_num_labels))
    

Percent of Correct Wikidata item in Candidates list: 0.5380392708846505
